Connect to MongoDB and excecute the aggregation

In [1]:
from pymongo import MongoClient
import json
from bson import ObjectId

def connect_to_db(db_name, collection_name):
    client = MongoClient("localhost:27017")
    db = client[db_name]
    collection = db[collection_name]
    return collection

def execute_aggregation(collection, pipeline):
    return list(collection.aggregate(pipeline))

def json_serializer(obj):
    if isinstance(obj, ObjectId):
        return str(obj)
    raise TypeError("Type not serializable")

road_to_search = input('Give int_name of road: ')

db_name = "Ptyxiaki"
collection_name = "Tripoli_Roads"
pipeline = [
    {
        '$unwind': {
            'path': '$features', 
            'preserveNullAndEmptyArrays': True
        }
    }, 
    {
        '$unwind': {
            'path': '$features.geometry.coordinates', 
            'preserveNullAndEmptyArrays': False
        }
    }, 
    {
        '$project': {
            'features.properties.@id': 1, 
            'features.properties.int_name': 1, 
            'features.geometry.coordinates': 1
        }
    }, 
    {
        '$sort': {
            'features.properties.int_name': 1
        }
    }, 
    {
        '$match': {
            'features.properties.int_name': road_to_search
        }
    }
]

collection = connect_to_db(db_name, collection_name)
output = execute_aggregation(collection, pipeline)

output_json = json.dumps(output, default=json_serializer, indent=4)

print(output_json)


[
    {
        "_id": "66fc2c336959dfdc0ebc6cec",
        "features": {
            "properties": {
                "@id": "way/153934648",
                "int_name": "Plataion"
            },
            "geometry": {
                "coordinates": [
                    22.3803427,
                    37.5205226
                ]
            }
        }
    },
    {
        "_id": "66fc2c336959dfdc0ebc6cec",
        "features": {
            "properties": {
                "@id": "way/153934648",
                "int_name": "Plataion"
            },
            "geometry": {
                "coordinates": [
                    22.3809864,
                    37.5202278
                ]
            }
        }
    },
    {
        "_id": "66fc2c336959dfdc0ebc6cec",
        "features": {
            "properties": {
                "@id": "way/153934648",
                "int_name": "Plataion"
            },
            "geometry": {
                "coordinates": [
                  

Set **output filename, output directory and output file**

In [2]:
# Get the output filename from the input file
output_filename = 'out_' + road_to_search + '.json'

# # Define the output directory
output_directory = '../Python_Scripts/ARKGis/OutputData/'

# # Full output path
output_file = f'{output_directory}out_{output_filename}'

print('output_filename: ' + output_filename, 
    'output_directory' + output_directory,
    'output_file: ' + output_file, 
    sep='\n')


output_filename: out_Plataion.json
output_directory../Python_Scripts/ARKGis/OutputData/
output_file: ../Python_Scripts/ARKGis/OutputData/out_out_Plataion.json


Reads a GeoJSON file, sorts its geographic features by their latitude, and then writes the sorted features to a new JSON file

In [3]:
# Function to sort the features by latitude
def sort_by_latitude(data):
    # Sort features by the latitude value, which is the second element in the "coordinates" list
    sorted_features = sorted(data['features'], key=lambda x: x['geometry']['coordinates'][1])
    
    # Replace the features with the sorted ones
    data['features'] = sorted_features
    return data

# Function to write the sorted JSON to a new file
def write_json_file(output_file, data):
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

# Main function to process the aggregation output directly
def process_data(output, output_file):
    # Convert the output to GeoJSON format directly
    geojson_data = convert_to_geojson(output)
    
    # Sort the data by latitude
    sorted_geojson_data = sort_by_latitude(geojson_data)

    myindex = 0
    # Add 'myindex' property for each feature
    for feature in sorted_geojson_data["features"]:
        myindex += 1
        feature["properties"]["myindex"] = myindex
        
    # Write the output GeoJSON to a file
    write_json_file(output_file, sorted_geojson_data)


Formats the GeoJSON to **ARKGis format** adding an incremental field for debunging (used in ARKGis)

In [4]:
# Function to convert the input JSON to GeoJSON format
def convert_to_geojson(data):
    geojson = {
        "type": "FeatureCollection",  # Root level type
        "features": []
    }

    for item in data:
        # Create the Feature object
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",  # Assuming coordinates are for a point. Adjust if needed
                "coordinates": item["features"]["geometry"]["coordinates"]
            },
            "properties": item["features"]["properties"]
        }

        # Append the feature to the FeatureCollection
        geojson["features"].append(feature)

    return geojson

# Function to write the output GeoJSON file
def write_geojson_file(output_file, geojson_data):
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(geojson_data, f, indent=2)
    print(f"GeoJSON saved to {output_file}")


In [5]:
# Run the script
process_data(output, output_file)
